In [1]:
import numpy as np
import io

import matplotlib.pyplot as plt

import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [2]:
file_path = r"H:\phd stuff\tidy3d\structures\End2EndFiles\filtered\lsu_0p8_filter.txt"
# file_path=r"H:\phd stuff\tidy3d\structures\End2EndFiles\Florescu LSU 14.3\ak4_1000_ends.dat"

In [3]:
def create_permittivity_grid_penlike(
    rod_endpoints,
    grid_size=128,
    minor_radius=0.1,        # circular radius in the pre-warp space (becomes ellipse after z-warp)
    aspect_ratio=None,        # global z scale factor s; ellipses have major/minor = s (projected)
    aspect_ratio_hole=None,        # global z scale factor s; ellipses have major/minor = s (projected)
    permittivity=3.42**2, #Si
    hole_minor_radius=0.0,   # inner circular radius in pre-warp space (same eccentricity after warp)
    box_size=None,
    *,
    progress_every=20,
    dinamic_radius=False,
    sigma_inner=None,
    sigma_outer=None,
    mean_inner=None,
    mean_outer=None,
    create_hole=False,
    use_radius_array=False,
    b_list=None,
    b_h_list=None,
    verbose=False

):
    """
    Build hollow 'pen-like' rods by:
      (1) creating RIGHT CIRCULAR cylinders in an unwarped space (x, y, z'),
      (2) applying a GLOBAL z warp: z = s * z'.

    IMPORTANT:
      - 'rod_endpoints' are given in FINAL (post-warp) world coords.
      - We unwarp them internally: z' = z / s.
      - Because the warp is GLOBAL, both shell and hole share the SAME ellipticity.
        You can set different radii (thickness), but not different ellipse ARs.

    Geometry:
      - Axis unit vector and clamping 0<=t<=L are computed in UNWARPED space.
      - Membership test is purely circular in UNWARPED cross-sections.
      - All ellipses are aligned with the projection of +z into each rod cross-plane.
        For rods parallel to z, the cross-section remains circular (expected).

    Returns:
      grid: (G,G,G) float32 array, ones background with 'permittivity' written inside rods.
    """
    rods = np.asarray(rod_endpoints, dtype=np.float32)  # scale to match original design
    G = int(grid_size)
    grid = np.ones((G, G, G), dtype=np.float32)

    dx = float(box_size) / G
    
    grid_coords = (np.arange(G, dtype=np.float32) + 0.5) * dx - np.float32(box_size / 2.0)
    # Debug: how many endpoints are outside the box?
    pts = rods.reshape(-1, 3)
    half = box_size / 2.0
    outside_mask = np.any((pts < -half) | (pts > half), axis=1)
    n_out = int(np.count_nonzero(outside_mask))
    if n_out > 0 and verbose:
        print(f"[warn] {n_out}/{pts.shape[0]} rod endpoints fall outside the box [-{half},{half}]^3.")

    # ----- radii setup (ensure arrays always exist) -----
    if not dinamic_radius:
        b_array  = np.full(len(rods), float(minor_radius), dtype=np.float32)
        b_h_array = np.full(len(rods), float(hole_minor_radius), dtype=np.float32)
    else:
        b_array  = np.random.normal(loc=mean_outer, scale=sigma_outer, size=len(rods)).astype(np.float32)
        b_h_array = np.random.normal(loc=mean_inner, scale=sigma_inner, size=len(rods)).astype(np.float32)
    
    if use_radius_array:
        print("Using provided b_list for outer radii.")
        if b_list is None:
            raise ValueError("b_list must be provided when use_radius_array=True.")
        if len(b_list) != len(rods):
            raise ValueError("b_list must have the same length as rod_endpoints.")
        b_array  = np.asarray(b_list, dtype=np.float32)
        if b_h_list is not None:
            b_h_array= np.asarray(b_h_list, dtype=np.float32)

    s = float(aspect_ratio) if aspect_ratio is not None else 1.0
    s_in = float(aspect_ratio_hole) if aspect_ratio_hole is not None else 1.0
    k_in  = s / s_in  # inner boundary z' scaling in UNWARPED space

    def idx_range_for_world(min_w, max_w, pad):
        lo = min_w - pad; hi = max_w + pad
        i0 = int(np.searchsorted(grid_coords, lo, side='left'))
        i1 = int(np.searchsorted(grid_coords, hi, side='right') - 1)
        i0 = max(i0, 0); i1 = min(i1, G - 1)
        if i1 < i0:
            mid = 0.5 * (min_w + max_w)
            i0 = i1 = max(min(int(np.searchsorted(grid_coords, mid, side='left')), G - 1), 0)
        return i0, i1

    for i_rod, rod in enumerate(rods):
        if progress_every and (i_rod % progress_every == 0):
            print(f"[postwarp] rod {i_rod} / {len(rods)}")

        b = b_array[i_rod]
        b_h = b_h_array[i_rod]
            
        use_hole = (b_h > 0.0) and (b_h < b)
        
        # pad in WORLD space: radius along directions with z-component can grow by up to s
        r_pad_world = b * max(1.0, s) + dx
        # FINAL (world) endpoints
        p1w = rod[:3].astype(np.float32)
        p2w = rod[3:].astype(np.float32)

        # UNWARP endpoints (z' = z / s) to build circular cylinder there
        p1u = p1w.copy(); p1u[2] = p1w[2] / s
        p2u = p2w.copy(); p2u[2] = p2w[2] / s

        vu = p2u - p1u
        L2u = float(np.dot(vu, vu))
        if L2u <= 0.0:
            continue
        Lu = float(np.sqrt(L2u))
        nu = vu / Lu  # axis in UNWARPED space

        # WORLD AABB expanded
        xmin, xmax = float(min(p1w[0], p2w[0])), float(max(p1w[0], p2w[0]))
        ymin, ymax = float(min(p1w[1], p2w[1])), float(max(p1w[1], p2w[1]))
        zmin, zmax = float(min(p1w[2], p2w[2])), float(max(p1w[2], p2w[2]))
        ix0, ix1 = idx_range_for_world(xmin, xmax, r_pad_world)
        iy0, iy1 = idx_range_for_world(ymin, ymax, r_pad_world)
        iz0, iz1 = idx_range_for_world(zmin, zmax, r_pad_world)

        xs = grid_coords[ix0:ix1+1]
        ys = grid_coords[iy0:iy1+1]
        zs = grid_coords[iz0:iz1+1]
        X, Y, Z = np.meshgrid(xs, ys, zs, indexing='ij')

        # Map WORLD coords to UNWARPED coords (x, y, z') with z' = z / s
        Zu = Z / s

        # Vector from p1 in UNWARPED space
        RXu = X - p1u[0]
        RYu = Y - p1u[1]
        RZu = Zu - p1u[2]

        # Axial coordinate and clamping in UNWARPED space
        tu = RXu * nu[0] + RYu * nu[1] + RZu * nu[2]
        mask_len = (tu >= 0.0) & (tu <= Lu)

        # Perpendicular distance in UNWARPED space (circular test)
        rX = RXu - tu * nu[0]
        rY = RYu - tu * nu[1]
        rZ = RZu - tu * nu[2]


        r2 = rX**2 + rY**2 + rZ**2
        outer_ok = r2 <= (b**2)


        if use_hole and create_hole:
            r2_inner = rX**2 + rY**2 + (k_in * rZ)**2
            inner_ok = r2_inner <= (b_h**2)
            final_mask = mask_len & outer_ok & (~inner_ok)
        else:
            final_mask = mask_len & outer_ok

        if np.any(final_mask):
            sub = grid[ix0:ix1+1, iy0:iy1+1, iz0:iz1+1]
            sub[final_mask] = permittivity
            grid[ix0:ix1+1, iy0:iy1+1, iz0:iz1+1] = sub


    grid_ff = grid.copy()
    grid_ff[grid_ff==1] = 0
    grid_ff[grid_ff>0] = 1
    ff = grid_ff.mean()

    return grid,b_array,ff


In [4]:
# Load the data from the string
rod_endpoints = np.loadtxt(file_path)

In [5]:
p1w = rod_endpoints[:,:3].astype(np.float32)
p2w = rod_endpoints[:,3:].astype(np.float32)
r_min, r_max =  0.1832, 0.262
# r_min, r_max =  0.1, 0.4
vectors = p2w - p1w
z_points = p1w[:,2]
t = (z_points - z_points.min()) / (z_points.max() - z_points.min())
t[t<=1/6]=0
t[(t>1/6) & (t<=2/6)]=1/5
t[(t>2/6) & (t<=3/6)]=2/5
t[(t>3/6) & (t<=4/6)]=3/5
t[(t>4/6) & (t<=5/6)]=4/5
t[t>5/6]=1

rads_minor = r_min + (r_max - r_min) * t
rads_minor = np.ones(z_points.shape)*0.235

In [6]:
#rads holes 
scaling_pixel = 0.26462/49.01 # The smaller radius of the elliptical cross-section
MEAN_INNER=0#2.12*4.03*scaling_pixel
SIGMA_INNER=8*scaling_pixel
ASPEC = 2.5
ASPEC_INNER = 4
rads_holes = np.abs(np.random.normal(loc=MEAN_INNER, scale=SIGMA_INNER, size=len(rod_endpoints)).astype(np.float32))
rads_holes[rads_holes*ASPEC_INNER>=0.2*ASPEC]=0
# rng = np.random.default_rng(42)  # optional seed
# mask = rng.random(rads_holes.shape) < 0.20   # True ~20% of the time
# rads_holes[mask] = 0

In [7]:
dir='./Structures_constant_r_holes_mean_zero'
os.makedirs(dir, exist_ok=True)

In [ ]:
# --- Parameters you can change ---
BOX_SIZE = 14.3*0.8  # Size of the simulation box in each dimension
GRID_SIZE = 512  # Resolution of the grid (e.g., 64, 128, 256)
# HOLE_MINOR_RADIUS = 0 # The smaller radius of the elliptical cross-section
PERM=3.2**2
MINOR_RADIUS = 0.1832 # The smaller radius of the elliptical cross-section
permittivity_grid,b_array,ff = create_permittivity_grid_penlike(rod_endpoints, grid_size=GRID_SIZE, 
                                                                aspect_ratio=ASPEC, progress_every=None,
                                                                aspect_ratio_hole=ASPEC_INNER,
                                                                permittivity=PERM,box_size=BOX_SIZE,
                                                                create_hole=True,use_radius_array=True
                                                                ,b_list=rads_minor,b_h_list=rads_holes,verbose=True
                                                                )
# AM.create_hdf5_from_dict({"epsilon":permittivity_grid},rf"./Structures/n_{np.sqrt(PERM):.2f}_ff_0.15_0.25.h5")
AM.create_hdf5_from_dict({"epsilon":permittivity_grid},rf"{dir}/n_{np.sqrt(PERM):.2f}_ff_{ff:.4f}.h5")
ff

[warn] 306/3306 rod endpoints fall outside the box [-5.720000000000001,5.720000000000001]^3.
Using provided b_list for outer radii.


0.2228683

: 